***
### Import of required libraries
***

In [1]:
from traffic.data import opensky

from traffic.core import Traffic
import pandas as pd

/home/mora/.conda/envs/traffic/lib/python3.10/site-packages/pyopensky/impala.py:17: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


***
### Import of flightplan data
***

In [2]:
flightplan_path = (
    "/store/Projects_CRM/RAD_paper/RAD_new_data/flightplans_complete.parquet"
)
flightplans = pd.read_parquet(flightplan_path)

***
### Download trajectory data per airport pair and direction
***

##### Function to remove invalid trajectories

In [3]:
def remove_invalid(flight):
    if (
        (len(flight.data) > 600)
        and (flight.data.altitude.max() > 10000)
        and (flight.cumulative_distance().data.cumdist.max() >= 10)
    ):
        return flight

##### London airports to consider

In [4]:
london_airports = [
    "EGLL",  # Heathrow
    "EGLC",  # London City
    "EGKK",  # Gatwick
]

##### Fetching of data

In [5]:
# Path where to store the trajectory data
base_path = "/store/Projects_CRM/RAD_paper/RAD_new_data/trajectories/"

# Iterate over london airports
for airport in london_airports:
    # Fetch Zurich to London
    print(f"processing Zurich to London {airport}")
    callsigns = flightplans.query(
        "origin == 'LSZH' and destination == @airport"
    ).callsign.unique()
    t = opensky.history(
        start="2024-05-01 00:00:00",
        stop="2024-10-16 00:00:00",
        callsign=callsigns,
    )
    if t is not None:
        t = t.clean_invalid().assign_id().eval(max_workers=20, desc="preprocessing")
        t = (
            t.iterate_lazy()
            .pipe(remove_invalid)
            .eval(max_workers=20, desc="removing invalid")
        )
        t.to_parquet(f"{base_path}zurich_to_london_{airport}.parquet")

    # Fetch London to Zurich
    print(f"processing London {airport} to Zurich")
    callsigns = flightplans.query(
        "origin == @airport and destination == 'LSZH'"
    ).callsign.unique()

    t = opensky.history(
        start="2024-05-01 00:00:00",
        stop="2024-10-16 00:00:00",
        callsign=callsigns,
    )
    if t is not None:
        t = t.clean_invalid().assign_id().eval(max_workers=20, desc="preprocessing")
        t = (
            t.iterate_lazy()
            .pipe(remove_invalid)
            .eval(max_workers=20, desc="removing invalid")
        )
        t.to_parquet(f"{base_path}london_to_zurich_{airport}.parquet")

processing Zurich to London EGLL


preprocessing:   0%|          | 0/2271 [00:00<?, ?it/s]

removing invalid:   0%|          | 0/2271 [00:00<?, ?it/s]

processing London EGLL to Zurich


preprocessing:   0%|          | 0/2614 [00:00<?, ?it/s]

...

preprocessing:   0%|          | 0/460 [00:00<?, ?it/s]

removing invalid:   0%|          | 0/460 [00:00<?, ?it/s]

processing London EGKK to Zurich


preprocessing:   0%|          | 0/751 [00:00<?, ?it/s]

removing invalid:   0%|          | 0/751 [00:00<?, ?it/s]